In [ ]:
import cv2
from super_gradients.common.object_names import Models
from super_gradients.training import models
import numpy as np
from multiprocessing import Pool, cpu_count

# Define o caminho do vídeo de entrada
VIDEO_PATH = "/kaggle/input/video/videoplayback (1).mp4"

# Carrega o modelo YOLO-NAS-L
model = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")

def process_frame(frame):
    """
    Processa um único quadro do vídeo.
    Retorna o quadro com as previsões desenhadas.
    """
    # Faz previsões no quadro
    predictions = model.predict(frame, conf=confidence_threshold)

    # Extrai informações das previsões apenas para a classe de pessoa
    for prediction in predictions:
        class_names = prediction.class_names
        labels = prediction.prediction.labels
        confidence = prediction.prediction.confidence
        bboxes = prediction.prediction.bboxes_xyxy

        for label, conf, bbox in zip(labels, confidence, bboxes):
            # Verifica se o rótulo é para uma pessoa
            if class_names[int(label)] == 'person' and conf >= confidence_threshold:
                # Desenha a caixa delimitadora e o rótulo no quadro
                xmin, ymin, xmax, ymax = map(int, bbox)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, f'Person: {conf:.2f}', (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame

# Define o limite de confiança inicial
confidence_threshold = 0.2

# Abre o arquivo de vídeo
cap = cv2.VideoCapture(VIDEO_PATH)

# Obtém a largura e a altura do quadro do vídeo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define o codec e cria o objeto VideoWriter
output_path = "/kaggle/working/output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

# Processa cada frame do vídeo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Processa o quadro
    frame_processed = process_frame(frame)

    # Escreve o quadro processado no vídeo de saída
    out.write(frame_processed)

# Libera os objetos de captura de vídeo e gravação
cap.release()
out.release()

print("Vídeo com previsões salvo com sucesso em:", output_path)


[2024-05-11 12:34:06] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-11 12:34:06] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l
[2024-05-11 12:34:06] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 12:34:07] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 12:34:08] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 12:34:09] INFO - pipelines.py - Fusing some of the mod